In [1]:
import numpy as np
import copy
from sklearn.neural_network import MLPClassifier
from numpy.random import *
from operator import itemgetter

In [2]:
def inputFieldData(text_path):
    f = open(text_path)
    line = f.readline()
    line_count = 1
    field_size = int(len(line) / 2)
    field = np.empty((0,field_size),int)
    agents_pos_list = [[6,1],[1,1],[5,6]]
    goal_pos_list = [[1,1],[5,6],[6,1]]
    while line:
        last_idx = line.find("\n")
        field_row = np.array([])
        for i in range(0,last_idx,2):
            field_row = np.append(field_row,int(line[i]))
        field = np.append(field,np.array([field_row]),axis=0)
        line = f.readline()
        line_count += 1
    f.close()
    return field,agents_pos_list,goal_pos_list

In [3]:
field,agents_pos_list,goal_pos_list = inputFieldData("FieldTest.txt")
print(field)
print("agents_pos_list = {0}".format(agents_pos_list))

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  1.  0.  1.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]
agents_pos_list = [[6, 1], [1, 1], [5, 6]]


In [4]:
field_row = len(field[0])
field_col = len(field)

In [5]:
alpha = 0.1
q_gamma = 0.9
now_pos_ground = 0

## aboutAction
action=0 : ↑
action=1 : →
action=2 : ↓
action=3 : ←

In [6]:
def jumpToNextPos(pos,action):
    if action == 0:
        next_pos = [pos[0]-1,pos[1]]
    elif action == 1:
        next_pos = [pos[0],pos[1]+1]
    elif action == 2:
        next_pos = [pos[0]+1,pos[1]]
    else:
        next_pos = [pos[0],pos[1]-1]
    return next_pos

In [7]:
def checkPosition(pos,action,field):
    r = action
    next_pos = jumpToNextPos(pos,action)
    while field[next_pos[0],next_pos[1]] == 1:
        r = int(rand()*4)
        while r == action:
            r = int(rand()*4)
        next_pos = jumpToNextPos(pos,r)
    return r

In [8]:
def buildNetwork(X_train,y_train):
    clf = MLPClassifier(solver="adam",random_state=1,max_iter=1000,hidden_layer_sizes=(20,50,10))
    print("X_train.shape = {0},y_train.shape={1}".format(X_train.shape,y_train.shape))
    print("y_train={0}".format(y_train))
    clf.fit(X_train,y_train)
    return clf

In [9]:
def initNetwork(field):
    X_train = np.asarray([field])
    X_train = X_train.reshape(1,field_row*field_col)
    y_train = np.array([[0]])
    clf = buildNetwork(X_train,y_train)
    return clf

In [10]:
def predictAction(clf,field):
    X_test = np.asarray(field).reshape(1,field_row*field_col)
    action = clf.predict(X_test)[0]
    if rand() < 0.3:
        action = int((action + rand() * 3 + 1)) % 4
    return action

In [11]:
def createTrainData(init_pos,field,goal_pos,clf=None):
    createFlag = False
    field_data = [copy.deepcopy(field)]
    action_data = []
    global now_pos_ground
    pos = init_pos
    for i in range(SEARCH_ROOT_MAX_LOOP):
        if clf != None:
            pred_action = predictAction(clf,field)
        else:
            pred_action = int(rand()*4)
        action = checkPosition(pos,pred_action,field)
        action_data.append(action)
        next_pos = jumpToNextPos(pos,action)
#         print("next_pos = {0}".format(next_pos))
        now_pos_ground = updateField(pos,next_pos,field)
        if next_pos == goal_pos:
#             printFieldData(field_data,action_data)
            createFlag = True
            break
        field_data.append(copy.deepcopy(field))
        pos = next_pos
    return field_data,action_data,createFlag

In [459]:
def printFieldData(field_data,action_data):
    print("------------------------------------------------")
    for i in range(len(field_data)):
        print(field_data[i])
        print(action_data[i])
    print("------------------------------------------------")

In [460]:
def updateField(pos,next_pos,field):
    field[pos[0],pos[1]] = now_pos_ground
    next_pos_ground = field[next_pos[0],next_pos[1]]
    field[next_pos[0],next_pos[1]] = 2
#     print(field)
#     print("\n")
    return next_pos_ground

In [461]:
SEARCH_ROOT_MAX_LOOP = 200 
def searchPath(agent_num):
    field,agents_pos_list,goal_pos_list = inputFieldData("FieldTest.txt")
    start_pos = agents_pos_list[agent_num]
    goal_pos = goal_pos_list[agent_num]
    print("init_field = \n{0}".format(field))
    clf = None
    path_count = 1e+7
    pathes = []
    for i in range(300):
        createFlag = False
        now_pos_ground = 0
        if (i+1) % 10 == 0:
            print("epoch = {0}".format(i+1))
        field_data,action_data,createFlag = createTrainData(start_pos,field,goal_pos,clf)
        if createFlag:
            np_field_data = np.asarray(field_data)
            np_action_data = np.asarray(action_data)
            if path_count > np_field_data.shape[0]-20:
                path_count = np_field_data.shape[0]
                clf = buildNetwork(np_field_data.reshape(np_field_data.shape[0],np_field_data.shape[1]*np_field_data.shape[2]),np_action_data)
                pathes.append([action_data,len(action_data)])
    return pathes

In [462]:
pathes_list = [searchPath(i) for i in range(len(agents_pos_list))]

init_field = 
[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  1.  0.  1.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]
X_train.shape = (15, 64),y_train.shape=(15,)
y_train=[0 0 0 2 0 0 1 0 1 3 2 3 1 3 0]
X_train.shape = (9, 64),y_train.shape=(9,)
y_train=[0 0 0 0 2 0 1 0 3]
X_train.shape = (13, 64),y_train.shape=(13,)
y_train=[0 0 0 0 1 3 2 2 1 3 0 0 0]
X_train.shape = (5, 64),y_train.shape=(5,)
y_train=[0 0 0 0 0]
X_train.shape = (11, 64),y_train.shape=(11,)
y_train=[0 0 1 3 0 0 1 3 1 0 3]
X_train.shape = (7, 64),y_train.shape=(7,)
y_train=[0 2 0 0 0 0 0]
X_train.shape = (5, 64),y_train.shape=(5,)
y_train=[0 0 0 0 0]
X_train.shape = (5, 64),y_train.shape=(5,)
y_train=[0 0 0 0 0]
X_train.shape = (7, 64),y_train.shape=(7,)
y_train=[0 0 2 0 0 0 0]
epoch = 10
X_train.shape = (5, 64),y_train.shape=(5,)
y_train=[0 0 0

X_train.shape = (13, 64),y_train.shape=(13,)
y_train=[0 0 0 0 1 0 2 0 2 0 2 0 3]
X_train.shape = (13, 64),y_train.shape=(13,)
y_train=[0 0 1 3 0 0 2 0 2 2 0 0 0]
X_train.shape = (9, 64),y_train.shape=(9,)
y_train=[0 0 0 0 2 0 1 3 0]
X_train.shape = (5, 64),y_train.shape=(5,)
y_train=[0 0 0 0 0]
X_train.shape = (9, 64),y_train.shape=(9,)
y_train=[0 0 1 1 3 3 0 0 0]
X_train.shape = (11, 64),y_train.shape=(11,)
y_train=[0 2 0 0 0 2 0 0 1 0 3]
epoch = 150
X_train.shape = (25, 64),y_train.shape=(25,)
y_train=[0 2 0 2 0 2 0 0 2 2 0 0 0 2 0 2 1 3 0 2 0 0 1 0 3]
X_train.shape = (39, 64),y_train.shape=(39,)
y_train=[0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 0 2 2 0 2 0 0 0 2 0 2 2 0 0 2 1 3 0 2 0 0 1
 0 3]
X_train.shape = (31, 64),y_train.shape=(31,)
y_train=[0 2 0 0 2 2 0 0 0 2 0 2 2 2 0 2 0 2 0 2 0 2 0 0 0 2 0 0 1 0 3]
X_train.shape = (19, 64),y_train.shape=(19,)
y_train=[0 2 0 2 0 2 0 0 0 2 0 0 1 0 1 1 3 3 3]
epoch = 160
X_train.shape = (7, 64),y_train.shape=(7,)
y_train=[0 0 1 3 0 0 0]
X_train.shape = 

X_train.shape = (5, 64),y_train.shape=(5,)
y_train=[0 0 0 0 0]
X_train.shape = (7, 64),y_train.shape=(7,)
y_train=[0 2 0 0 0 0 0]
X_train.shape = (7, 64),y_train.shape=(7,)
y_train=[0 0 2 0 0 0 0]
X_train.shape = (5, 64),y_train.shape=(5,)
y_train=[0 0 0 0 0]
X_train.shape = (11, 64),y_train.shape=(11,)
y_train=[0 2 0 0 2 2 0 0 0 0 0]
X_train.shape = (19, 64),y_train.shape=(19,)
y_train=[0 2 0 2 0 2 0 2 0 2 0 2 0 0 0 2 0 0 0]
X_train.shape = (9, 64),y_train.shape=(9,)
y_train=[0 2 0 2 0 0 0 0 0]
epoch = 280
X_train.shape = (11, 64),y_train.shape=(11,)
y_train=[0 2 0 0 2 0 0 0 1 3 0]
X_train.shape = (5, 64),y_train.shape=(5,)
y_train=[0 0 0 0 0]
X_train.shape = (9, 64),y_train.shape=(9,)
y_train=[0 2 0 0 0 0 2 0 0]
X_train.shape = (21, 64),y_train.shape=(21,)
y_train=[0 0 1 1 1 0 0 0 2 0 2 0 2 0 2 0 3 3 1 3 3]
X_train.shape = (29, 64),y_train.shape=(29,)
y_train=[0 0 1 3 2 0 0 2 2 0 1 1 1 0 0 0 2 0 3 3 1 3 1 1 2 0 3 3 3]
X_train.shape = (29, 64),y_train.shape=(29,)
y_train=[0 2 0 0 2 0 

X_train.shape = (17, 64),y_train.shape=(17,)
y_train=[1 1 1 2 2 2 3 1 3 3 1 1 2 0 2 1 1]
X_train.shape = (31, 64),y_train.shape=(31,)
y_train=[1 1 1 2 0 2 0 3 1 2 0 3 1 3 1 2 0 2 2 2 3 1 3 1 0 2 2 1 3 1 1]
X_train.shape = (19, 64),y_train.shape=(19,)
y_train=[1 1 1 2 0 2 0 2 2 0 2 2 2 1 3 3 1 1 1]
X_train.shape = (15, 64),y_train.shape=(15,)
y_train=[1 1 1 2 2 2 3 1 2 0 0 2 2 1 1]
X_train.shape = (19, 64),y_train.shape=(19,)
y_train=[2 2 2 0 0 2 2 1 1 3 1 1 0 0 2 2 2 1 1]
epoch = 110
X_train.shape = (15, 64),y_train.shape=(15,)
y_train=[2 0 2 1 3 0 2 2 2 1 1 1 2 1 1]
epoch = 120
X_train.shape = (11, 64),y_train.shape=(11,)
y_train=[2 0 2 2 2 1 1 1 2 1 1]
X_train.shape = (13, 64),y_train.shape=(13,)
y_train=[2 2 2 1 3 1 3 1 1 1 2 1 1]
X_train.shape = (23, 64),y_train.shape=(23,)
y_train=[2 1 0 3 2 1 3 1 0 3 2 1 3 2 2 1 3 1 1 1 2 1 1]
X_train.shape = (13, 64),y_train.shape=(13,)
y_train=[1 3 2 2 2 0 2 1 1 1 2 1 1]
X_train.shape = (27, 64),y_train.shape=(27,)
y_train=[1 3 2 2 0 2 2 0 2 1 

X_train.shape = (44, 64),y_train.shape=(44,)
y_train=[3 3 3 0 3 1 2 0 3 3 1 3 1 3 1 3 1 3 1 3 1 3 1 1 1 3 3 1 3 3 1 1 3 3 1 3 1
 3 1 3 1 3 2 2]
X_train.shape = (24, 64),y_train.shape=(24,)
y_train=[3 3 1 3 3 0 3 3 1 1 3 3 1 3 1 3 1 1 3 3 1 3 2 2]
epoch = 10
X_train.shape = (14, 64),y_train.shape=(14,)
y_train=[3 3 3 0 2 0 3 3 1 3 1 3 2 2]
X_train.shape = (16, 64),y_train.shape=(16,)
y_train=[3 3 3 0 1 2 3 2 0 1 0 3 3 3 2 2]
X_train.shape = (18, 64),y_train.shape=(18,)
y_train=[3 1 3 3 1 1 3 3 3 2 0 2 0 0 3 3 2 2]
epoch = 20
X_train.shape = (28, 64),y_train.shape=(28,)
y_train=[0 2 3 1 3 3 3 1 3 2 0 2 0 1 3 2 0 1 3 1 3 0 3 3 2 0 2 2]
X_train.shape = (18, 64),y_train.shape=(18,)
y_train=[3 3 3 1 3 1 0 3 3 3 2 0 2 0 2 0 2 2]
X_train.shape = (10, 64),y_train.shape=(10,)
y_train=[3 3 0 3 3 3 2 0 2 2]
X_train.shape = (14, 64),y_train.shape=(14,)
y_train=[3 3 0 3 2 1 0 2 0 3 3 3 2 2]
X_train.shape = (26, 64),y_train.shape=(26,)
y_train=[3 3 0 2 0 3 3 3 0 0 0 1 3 1 3 1 2 3 0 1 3 2 2 2 2 2]
X_t

In [468]:
for (i,pathes)  in enumerate(pathes_list):
    print("agents_num = {0}".format(i+1))
    print("pathes = ")
    for path in pathes:
        print(path)
    print("\n")

#modify path
for (i,pathes) in enumerate(pathes_list):
    for path_array in pathes:
        path = path_array[0]
        del_flag = True
        print("before modifying path : \n{0}".format(path))
        while del_flag == True:
            del_flag = False
            for i in range(len(path)):
                if i+1 < len(path):
                    if path[i]%2 == path[i+1]%2 and not(path[i] == path[i+1]):
                        del path[i+1]
                        del path[i]
                        del_flag == True
        print("after modifying path: \n{0}".format(path))
        path_array[1] = len(path)

agents_num = 1
pathes = 
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0, 0, 0], 5]
[[0, 0, 0

before modifying path : 
[3, 3, 3, 0, 3, 3, 2, 2]
after modifying path: 
[3, 3, 3, 0, 3, 3, 2, 2]
before modifying path : 
[3, 3, 3, 0, 3, 3, 2, 2]
after modifying path: 
[3, 3, 3, 0, 3, 3, 2, 2]
before modifying path : 
[3, 3, 0, 3, 3, 3, 2, 2]
after modifying path: 
[3, 3, 0, 3, 3, 3, 2, 2]
before modifying path : 
[3, 3, 0, 3, 3, 3, 2, 2]
after modifying path: 
[3, 3, 0, 3, 3, 3, 2, 2]
before modifying path : 
[3, 3, 3, 0, 3, 3, 2, 2]
after modifying path: 
[3, 3, 3, 0, 3, 3, 2, 2]
before modifying path : 
[3, 3, 0, 3, 3, 3, 2, 2]
after modifying path: 
[3, 3, 0, 3, 3, 3, 2, 2]
before modifying path : 
[3, 3, 0, 3, 3, 3, 2, 2]
after modifying path: 
[3, 3, 0, 3, 3, 3, 2, 2]
before modifying path : 
[3, 3, 0, 3, 2, 1, 0, 3, 3, 3, 2, 2]
after modifying path: 
[3, 3, 0, 3, 2, 1, 0, 3, 3, 3, 2, 2]
before modifying path : 
[3, 3, 0, 0, 0, 0, 3, 3, 3, 2, 2, 2, 2, 2]
after modifying path: 
[3, 3, 0, 0, 0, 0, 3, 3, 3, 2, 2, 2, 2, 2]
before modifying path : 
[3, 3, 3, 0, 1, 0, 0, 0, 3, 3, 2,

In [467]:
#sort
sorted_pathes_list = []
for pathes_array in pathes_list:
    pathes_array.sort(key=itemgetter(1))
    pathes = []
    for path_data in pathes_array:
        path = path_data[0]
#         print("path = {0}".format(path))
        pathes.append(path)
    sorted_pathes_list.append(pathes)
# for i in range(len(sorted_pathes_list[0])):
#     print(sorted_pathes_list[0][i])

In [438]:
def checkState(pos,next_pos,previous_field,field):
    if field[next_pos[0],next_pos[1]] == 1 or field[next_pos[0],next_pos[1]] == 2:
        #if next_pos is reserved
        return False
    elif previous_field[next_pos[0],next_pos[1]] == 2:
        # if the agent cross other agent
        return False
    else:
        return True

In [439]:
def checkReservedAgentPath(pos):
    for (i,root_pos) in enumerate (root_pos_list):
        agent_start_pos = agents_start_pos_list[i]
#         print("agent_start_pos = {0}".format(agent_start_pos))
#         print("root_pos = {0}".format(root_pos))
        for other_agent_pos in root_pos:
            if other_agent_pos == pos:
                return False
    return True

In [440]:
def updateFieldUsingPath(start_pos,pathes):
    for i in range(len(pathes)):
        path = pathes[-(i+1)]
        pos = start_pos
        root_pos = []
        for j in range(len(path)):
            append_flag = False
            roop_flag = False
            next_pos = jumpToNextPos(pos,path[j])
            if j < len(field_data):
                field_copy = field_data[j]
            else:
                field_copy = copy.deepcopy(field_data[-1])
                append_flag = True
            if j == 0:
                previous_field = field_data[0]
            else:
                previous_field = field_data[j-1]
            if checkState(pos,next_pos,previous_field,field_copy):
                field_copy[next_pos[0],next_pos[1]] = 2
                field_copy[pos[0],pos[1]] = 0
                root_pos.append(next_pos)
                if append_flag:
                    field_data.append(field_copy)
            else:
                print("collapse : {0}".format(next_pos))
                for (k,pos) in enumerate(root_pos):
                    field_data[k][pos[0],pos[1]] = 0
                roop_flag = True
                break
            pos = next_pos
            print(field_copy)
            print("")
            
        check_reserved_agent_path_flag = False
        if roop_flag == False:
            if checkReservedAgentPath(pos) == False:
                check_reserved_agent_path_flag = True
                for (k,pos) in enumerate(root_pos):
                    field_data[k][pos[0],pos[1]] = 0
            else:
                right_path = path
                if len(right_path) < len(field_data):
                    for j in range(len(field_data)):
                        if j < len(right_path):
                            continue
                        else:
                            field_data[j][pos[0],pos[1]] = 2
                            print(field_data[j])
                return path,root_pos
    if i+1 == len(pathes):
        print("-----------------------------------search fault--------------------------------")
    return path,root_pos

In [441]:
# field,agents_pos_list,goal_pos_list = inputFieldData("singleAgentField2.txt")
# agents_start_pos_list = copy.deepcopy(agents_pos_list)
# field_data = [copy.deepcopy(field)]
# right_path_list = []
# root_pos_list = []
# for (i,pathes) in enumerate (sorted_pathes_list):
#     right_path,root_pos = updateFieldUsingPath(agents_pos_list[i],pathes)
#     print("right_path{0} = {1}".format(i+1,right_path))
#     right_path_list.append(right_path)
#     root_pos_list.append(root_pos)
    
# step_num = max(len(right_path_list[0]),len(right_path_list[1]),len(right_path_list[2]))
# field_data = [copy.deepcopy(field) for i in range(step_num)]
# agents_num = len(agents_pos_list)
# init_field = copy.deepcopy(field)
# for i in range(agents_num):
#     init_field[agents_pos_list[i][0],agents_pos_list[i][1]] = 2
#     print("agent{0}_start_pos = {1}".format(i+1,agents_pos_list[i]))
# for i in range(step_num):
#     field = field_data[i]
#     for j in range(agents_num):
#         if i < len(right_path_list[j]):
#             next_pos = jumpToNextPos(agents_pos_list[j],right_path_list[j][i])
#             field[next_pos[0],next_pos[1]] = 2
#             agents_pos_list[j] = next_pos
#         else:
#             field[agents_pos_list[j][0],agents_pos_list[j][1]] = 2
# field_data.insert(0,init_field)

In [442]:
def checkNextPos(pos,next_pos,field):
    if pos == next_pos:
        return True
    elif field[next_pos[0],next_pos[1]] == 2:
        return False
    else:
        return True

In [469]:
field,agents_pos_list,goal_pos_list = inputFieldData("FieldTest.txt")
init_field = copy.deepcopy(field)
for i in range(len(agents_pos_list)):
    init_field[agents_pos_list[i][0],agents_pos_list[i][1]] = 2
right_path_list = []
root_pos_list = []
SEARCH_RIGHT_PATHES_ROOP = 200
agents_num = len(agents_pos_list)
agents_path_candidate = [sorted_pathes_list[i][0] for i in range(agents_num)]
cnt = [0,0,0]
for i in range(SEARCH_RIGHT_PATHES_ROOP):
    field,agents_pos_list,goal_pos_list = inputFieldData("FieldTest.txt")
    create_next_step_flag = False
    roop_flag = False
    field_data = [copy.deepcopy(field)]
    max_path_num = max(len(agents_path_candidate[0]),len(agents_path_candidate[1]),len(agents_path_candidate[2]))
    pos_list = copy.deepcopy(agents_pos_list)
    for j in range(max_path_num):
        #each time step
        if j < len(field_data):
            field = field_data[j]
        else:
            field = copy.deepcopy(field_data[j-1])
            create_next_step_flag = True
        for k in range(len(agents_path_candidate)):
            #each agent
            pos = pos_list[k]
            if j < len(agents_path_candidate[k]):
                action = agents_path_candidate[k][j]
                next_pos = jumpToNextPos(pos,action)
            else:
                next_pos = pos
            if checkNextPos(pos,next_pos,field):
                field[pos[0],pos[1]] = 0
                field[next_pos[0],next_pos[1]] = 2
                pos_list[k] = next_pos
#                 print(field)
#                 print("")
            else:
                roop_flag = True
                break
        if roop_flag:
            print("search_fault")
            if i == SEARCH_RIGHT_PATHES_ROOP-1:
                print("all search is fault")
            r = int(rand() * agents_num)
            roop_cnt = 0
            while cnt[r] >= len(sorted_pathes_list[r]):
                roop_cnt += 1
                if roop_cnt > 100:
                    break
                r = int(rand() * agents_num)
            agents_path_candidate[r] = sorted_pathes_list[r][cnt[r]]
            cnt[r] += 1
            break
#         else:
#             for i in range(agents_num):
#                 print(agents_path_candidate[i])
        if create_next_step_flag:
            field_data.append(copy.deepcopy(field))
field_data.insert(0,init_field)
for field in field_data:
    print(field)
    print("")

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  2.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  2.  1.]
 [ 1.  2.  1.  0.  1.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  2.  0.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  1.  0.  1.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.]
 [ 1.  2.  1.  0.  0.  2.  0.  1.]
 [ 1.  0.  1.  0.  1.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  2.  0.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  0.  1.  1.  0.  1.  0.  1.]
 [ 1.  2.  0.  0.  0.  1.  0.  1.]
 [ 1.  0.  1.  0.  2.  0.  0.  1.]
 [ 1.  0.  1.  0.  1.  1.  0.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]

[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  0.  0.  2.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  1.  0.  1.]
 [ 1.  2.  1.  1.  0.  1.  0.  1.]
 [ 1.  0.  0. 

In [470]:
# for field in field_data:
#     print(field)
#     print("\n")

In [474]:
field_data_string =  []
for field in field_data:
    field_string = []
    for i in range(len(field)):
        field_string_col = []
        for j in range(len(field[0])):
            if field[i][j] == 0:
                field_string_col.append("｜")
            elif field[i][j] == 1:
                field_string_col.append("■")
            elif field[i][j] == 2:
                field_string_col.append("□")
        field_string.append(field_string_col)
    field_data_string.append(field_string)

In [475]:
for i in range(len(field_data_string)):
    new_field = field_data_string[i]
    for row in new_field:
        print(row)
    print("----------")

['■', '■', '■', '■', '■', '■', '■', '■']
['■', '□', '｜', '｜', '｜', '■', '｜', '■']
['■', '｜', '｜', '■', '｜', '■', '｜', '■']
['■', '｜', '■', '■', '｜', '■', '｜', '■']
['■', '｜', '｜', '｜', '｜', '■', '｜', '■']
['■', '｜', '■', '｜', '｜', '｜', '□', '■']
['■', '□', '■', '｜', '■', '■', '｜', '■']
['■', '■', '■', '■', '■', '■', '■', '■']
----------
['■', '■', '■', '■', '■', '■', '■', '■']
['■', '｜', '□', '｜', '｜', '■', '｜', '■']
['■', '｜', '｜', '■', '｜', '■', '｜', '■']
['■', '｜', '■', '■', '｜', '■', '｜', '■']
['■', '｜', '｜', '｜', '｜', '■', '｜', '■']
['■', '□', '■', '｜', '｜', '□', '｜', '■']
['■', '｜', '■', '｜', '■', '■', '｜', '■']
['■', '■', '■', '■', '■', '■', '■', '■']
----------
['■', '■', '■', '■', '■', '■', '■', '■']
['■', '｜', '｜', '□', '｜', '■', '｜', '■']
['■', '｜', '｜', '■', '｜', '■', '｜', '■']
['■', '｜', '■', '■', '｜', '■', '｜', '■']
['■', '□', '｜', '｜', '｜', '■', '｜', '■']
['■', '｜', '■', '｜', '□', '｜', '｜', '■']
['■', '｜', '■', '｜', '■', '■', '｜', '■']
['■', '■', '■', '■', '■', '■', '■',

In [476]:
print(len(field_data))

10


In [478]:
for right_path in right_path_list:
    print(right_path)